# ALGORITMO DE IA-4: REDE NEURAL CONVOLUCIONAL (CNN)

ENUNCIADO:  
Implementar uma Rede Neural Convolucional (CNN) usando as bibliotecas TensorFlow e Keras

# CASO 03: DATASET CIFAR

CIFAR-10 é um conjunto de dados que consiste em 60'000 imagens coloridas de 32x32 distribuidas em 10 classes, 6'000 imagens por classe. São 50'000 imagens no conjunto de treinamento e 10'000 imagens para teste

1) Importar livrarías/formatear dataset

In [60]:
#importar livrarías
import pandas as pd
import numpy as np
import keras
from keras.datasets import cifar10
from keras.utils import np_utils
from scipy.misc import toimage

In [61]:
#carregar conjunto de treinamento/teste
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

2) FASE DE PREPROCESSAMENTO

In [64]:
#Aplicar redimensionamento sobre os dados para obter uma matriz [#samples] x [width] x [weight] x [channels]

#conjunto de treinamento
x_train = X_train.reshape(X_train.shape[0], 32, 32, 3)

#conjunto de teste
x_test = X_test.reshape(X_test.shape[0], 32, 32, 3)

In [65]:
#converter os tipos de dado para float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [66]:
#standarização dos dados: x_std = (x-ux)/std(x)
x_train = (x_train - np.mean(x_train))/np.std(x_train)
x_test = (x_test - np.mean(x_test))/np.std(x_test)

In [67]:
#informação sobre a matriz de caraterísticas:
dim_inputs = x_train.shape[2] * x_train.shape[3]
print('#Variáveis na matriz de caraterísticas: ', dim_inputs)
print('#Dim. Matriz de Caraterísticas - train: ', x_train.shape)
print('#Dim. Matriz de Caraterísticas - test: ', x_test.shape)

#Variáveis na matriz de caraterísticas:  96
#Dim. Matriz de Caraterísticas - train:  (50000, 32, 32, 3)
#Dim. Matriz de Caraterísticas - test:  (10000, 32, 32, 3)


In [68]:
#visualizar saídas/classes
print('Output train: ', np.unique(Y_train))

#transformar o vetor de classes em uma matriz binária de classes [0...9]
y_train = keras.utils.to_categorical(Y_train, len(np.unique(Y_train)))
y_test = keras.utils.to_categorical(Y_test, len(np.unique(Y_test)))

#informação sobre a matriz de saídas: 
dim_outputs = len(np.unique(Y_train))
print('Classes possíveis: ', dim_outputs)
print('Dim. Matriz de Saídas - train: ', y_train.shape)
print('Dim. Matriz de Saídas - test: ', y_test.shape)

Output train:  [0 1 2 3 4 5 6 7 8 9]
Classes possíveis:  10
Dim. Matriz de Saídas - train:  (50000, 10)
Dim. Matriz de Saídas - test:  (10000, 10)


3) FASE DE APRENDIZADO

In [69]:
#importar livrarías
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization

Define-se a seguinte arquitetura de Rede Neural Convolucional para o CIFAR-10:

![title](img/cifar2.png)

![title](img/cifar3.png)

In [70]:
#criar arquitetura do modelo

#inicializar modelo
classifier = Sequential()

#1° Camada Convolucional
classifier.add(Convolution2D(filters = 32, kernel_size = (5, 5), strides = (1, 1), input_shape = (32, 32, 3), 
                             padding = 'same', activation = 'relu'))
#1° Camada Max-Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

#2° Camada Convolucional
classifier.add(Convolution2D(filters = 32, kernel_size = (5, 5), strides = (1, 1)
                             padding = 'same', activation = 'relu'))
#Normalização em Batch
classifier.add(BatchNormalization())

#2° Camada Max-Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

#Dropout
classifier.add(Dropout(0.3))

#Flattening
classifier.add(Flatten())

#Rede Neural Fully-connected
classifier.add(Dense(units = 240, activation = 'relu'))
classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dense(units = dim_outputs, activation = 'softmax'))

#Compilar CNN
classifier.compile(optimizer = 'adagrad', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [72]:
#treinamento do modelo
classifier.fit(x_train, y_train, batch_size = 200, epochs = 10, validation_data = (x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 385s 8ms/step - loss: 1.2213 - acc: 0.5657 - val_loss: 1.2187 - val_acc: 0.5739
Epoch 2/10
50000/50000 [==============================] - 407s 8ms/step - loss: 1.0486 - acc: 0.6282 - val_loss: 1.0239 - val_acc: 0.6298
Epoch 3/10
50000/50000 [==============================] - 402s 8ms/step - loss: 0.9360 - acc: 0.6692 - val_loss: 0.9468 - val_acc: 0.6644
Epoch 4/10
50000/50000 [==============================] - 414s 8ms/step - loss: 0.8565 - acc: 0.6961 - val_loss: 0.9129 - val_acc: 0.6758
Epoch 5/10
50000/50000 [==============================] - 432s 9ms/step - loss: 0.7887 - acc: 0.7225 - val_loss: 0.9017 - val_acc: 0.6870
Epoch 6/10
50000/50000 [==============================] - 414s 8ms/step - loss: 0.7352 - acc: 0.7403 - val_loss: 0.8701 - val_acc: 0.6968
Epoch 7/10
50000/50000 [==============================] - 411s 8ms/step - loss: 0.6856 - acc: 0.7587 - val_loss: 0.9017 - 

4) FASE DE AVALIAÇÃO

In [74]:
#cálculo da acurácia no conjunto de treino
score_train = classifier.evaluate(x_train, y_train)
print('Acurácia-train: ', 100*score_train[1], '%')

50000/50000 [==============================] - 333s 7ms/step
Acurácia-train:  86.194 %


In [73]:
#cálculo da acurácia no conjunto de teste
score_test = classifier.evaluate(x_test, y_test)
print('Acurácia-test: ', 100*score_test[1], '%')

10000/10000 [==============================] - 61s 6ms/step
Acurácia-test:  72.55 %


5) CONCLUSÕES:  
- Acurácia no conjunto de treinamento: 86.19%
- Acurácia no conjunto de teste: 72.55%
- O desempenho do modelo CNN pode ser melhorado fazendo hypeparameter tuning, 
porém recomenda-se melhor hardware para processamento computacional (GPUs)